<h1>Creating Geography lookup tables</h1>

This file creates a postcode look up using NSPL.<br>
This output can be be joined to tables read in using area/district/sector. <br>
Where a postcode crosses regional boundries a sclaing factor is applied. 
This is to be used when calculating the number of cardholders, spend etc.

In [ ]:
project_path = "/home/jupyter"
import sys
sys.path.append(project_path)

import numpy as np
import pandas as pd

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t
from google.cloud import bigquery

In [ ]:
client = bigquery.Client()

In [ ]:
sql_nspl = """SELECT *
FROM ons-fintrans-analysis-prod.fin_wip_notebook.nspl_postal_level_lookup"""

df_nspl = bq.read_bq_table_sql(client, sql_nspl)

In [ ]:
#POSTAL AREA
df_postal_area = df_nspl[['pcds','postal_area','region', 'country']].groupby(['postal_area', 'country','region'],as_index=False).count().reset_index()
df_postal_area['area_sf'] = df_postal_area['pcds'] / df_postal_area.groupby('postal_area')['pcds'].transform('sum')
df_postal_area = df_postal_area[['postal_area', 'area_sf', 'region', 'country']]

In [ ]:
#POSTAL DISTRICT
df_postal_district = df_nspl[['pcds','postal_district','region', 'country']].groupby(['postal_district', 'country','region'],as_index=False).count().reset_index()
df_postal_district['district_sf'] = df_postal_district['pcds'] / df_postal_district.groupby('postal_district')['pcds'].transform('sum')
df_postal_district = df_postal_district[['postal_district', 'district_sf', 'region', 'country']]

In [ ]:
#POSTAL SECTOR
df_postal_sector = df_nspl[['pcds','postal_sector','region', 'country']].groupby(['postal_sector', 'country','region'],as_index=False).count().reset_index()
df_postal_sector['sector_sf'] = df_postal_sector['pcds'] / df_postal_sector.groupby('postal_sector')['pcds'].transform('sum')
df_postal_sector = df_postal_sector[['postal_sector', 'sector_sf', 'region', 'country']]

In [ ]:
#ALL
df_postal_all = df_nspl[['postal_area','postal_district', 'postal_sector', 'region', 'country']]
df_postal_all = df_postal_all[df_postal_all['region'].notna()]
df_postal_all = df_postal_all.drop_duplicates()
df_postal_all = pd.merge(df_postal_all, df_postal_area, how="left", on=['postal_area', 'region', 'country'])
df_postal_all = pd.merge(df_postal_all, df_postal_district, how="left", on=['postal_district', 'region', 'country'])
df_postal_all = pd.merge(df_postal_all, df_postal_sector, how="left", on=['postal_sector', 'region', 'country'])

In [ ]:
#df_postal_all.to_csv("~/fintrans_toolbox/data/NSPL_location_level_lookup.csv", index=False)

In [ ]:
#saves dataframe as big query table
#to replace dataframe, change if_exists = 'replace'
df_postal_all.to_gbq('ons-fintrans-analysis-prod.fin_wip_notebook.nspl_postal_level_scaling_factors', project_id=None, chunksize=None, reauth=False, 
                 if_exists='replace', auth_local_webserver=True, table_schema=None, 
                 location=None, progress_bar=True, credentials=None)